In [1]:
import torch

In [2]:
class MyReLU(torch.autograd.Function):
    
    # 정적 메소드 --> 인스턴스화 하지 않고 호출 가능
    # 클래스 내부 참조 불가
    
    @staticmethod
    def forward(ctx, input):
        #        ctx : context object       
        #  역전파를 위한 정보를 저장(cache)        
        ctx.save_for_backward(input)
        return input.clamp(min = 0)
    
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone() # relu --> gradient 그대로
        grad_input[input < 0] = 0 # 0보다 작으면 0
        return grad_input

In [3]:
dtype = torch.float
device = torch.device('cpu')

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

In [4]:
learning_rate = 1e-6
for t in range(501):
    relu = MyReLU.apply
    
    #forward
    y_pred = relu(x.mm(w1)).mm(w2)
    
    #loss
    loss = (y_pred - y).pow(2).sum()
    if t % 10 == 0:
        print(t, loss.item())
    
    #backward
    loss.backward()
    
    #update weight
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.gr

0 39904616.0
10 2747715.0
20 619902.5625
30 508104.375
40 523631.75
50 3422299.0
60 13845286.0
70 4.324249147418751e+27
80 nan
90 nan
100 nan
110 nan
120 nan
130 nan
140 nan
150 nan
160 nan
170 nan
180 nan
190 nan
200 nan
210 nan
220 nan
230 nan
240 nan
250 nan
260 nan
270 nan
280 nan
290 nan
300 nan
310 nan
320 nan
330 nan
340 nan
350 nan
360 nan
370 nan
380 nan
390 nan
400 nan
410 nan
420 nan
430 nan
440 nan
450 nan
460 nan
470 nan
480 nan
490 nan
500 nan
